In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-03/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.tail())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di jalur /auto_weather_stat/id-03/data.")

                 dew humidity pressure temperature   timestamp  volt  humi  \
1740279793  27.25532    82.39  1012.56        30.6  1740279793  4.13   NaN   
1740279853  26.98808    80.83  1012.54       30.66  1740279853  4.13   NaN   
1740279913  26.99584    80.36  1012.45       30.77  1740279913  4.13   NaN   
1740279973  27.06351    80.68  1012.45       30.77  1740279973  4.13   NaN   
1740280033   27.3701    82.05  1012.44       30.79  1740280033  4.12   NaN   

            pres  temp  
1740279793   NaN   NaN  
1740279853   NaN   NaN  
1740279913   NaN   NaN  
1740279973   NaN   NaN  
1740280033   NaN   NaN  


In [10]:
df.tail(10)

,dew,humidity,pressure,temperature,timestamp,volt,humi,pres,temp
1740279493,27.33236,84.73,1012.59,30.19,1740279493,4.13,NaN,NaN,NaN
1740279553,27.35742,84.66,1012.58,30.23,1740279553,4.13,NaN,NaN,NaN
1740279613,27.19613,83.05,1012.53,30.4,1740279613,4.13,NaN,NaN,NaN
1740279673,27.28501,82.77,1012.56,30.55,1740279673,4.13,NaN,NaN,NaN
1740279733,27.14435,82.09,1012.6,30.55,1740279733,4.13,NaN,NaN,NaN
1740279793,27.25532,82.39,1012.56,30.6,1740279793,4.13,NaN,NaN,NaN
1740279853,26.98808,80.83,1012.54,30.66,1740279853,4.13,NaN,NaN,NaN
1740279913,26.99584,80.36,1012.45,30.77,1740279913,4.13,NaN,NaN,NaN
1740279973,27.06351,80.68,1012.45,30.77,1740279973,4.13,NaN,NaN,NaN
1740280033,27.3701,82.05,1012.44,30.79,1740280033,4.12,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 716797 entries, 1702296831 to 1740280033
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   dew          716797 non-null  object 
 1   humidity     716793 non-null  object 
 2   pressure     716793 non-null  object 
 3   temperature  716793 non-null  object 
 4   timestamp    716797 non-null  object 
 5   volt         716797 non-null  object 
 6   humi         4 non-null       float64
 7   pres         4 non-null       float64
 8   temp         4 non-null       float64
dtypes: float64(3), object(6)
memory usage: 54.7+ MB


In [6]:
df = df[(df['temperature'] >= 10) & 
                      (df['humidity'] >= 0) & 
                      (df['pressure'] >= 1000) & 
                      (df['dew'] >= 10) &
                      (df['volt'] >= 0)]

TypeError: '>=' not supported between instances of 'str' and 'int'

In [20]:
# Membuat DataFrame awal
data = pd.DataFrame(df)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Membuat rentang waktu yang lengkap dengan interval per menit
complete_range = pd.date_range(start=data.index.min(), end=df.index.max(), freq='T')

# Reindexing dengan rentang waktu lengkap, variabel lain akan terisi NaN
df = df.reindex(complete_range)

# Menampilkan hasil
print(df.head(15))

/tmp/ipykernel_392/737626841.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'])


DateParseError: day is out of range for month: 0, at position 0

In [8]:
# Plot line chart
df.plot(y=['dew', 'humidity', 'pressure', 'temperature', 'volt'], subplots=True, figsize=(10, 15))
plt.show()

TypeError: no numeric data to plot

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>